In [1]:
from datetime import date
from datetime import datetime
from datetime import timedelta
import os
import subprocess
import urllib
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib import style

/tmp/ipykernel_3088/3305657379.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
today = datetime.today().strftime('%Y-%m-%d')
beginning = '2020-03-19'
if not(os.path.exists('Covid-Study-Data')):
    cmd = 'mkdir ' + os.path.join(os.getcwd(),'Covid-Study-Data')
    print(cmd)
    (status, output) = subprocess.getstatusoutput(cmd)    # try to make a new directory
    print("STATUS:\n", status,"OUTPUT:\n", output)                            # 0 implies error free!
else:
    print("'{}' already exists".format(os.path.join(os.getcwd(), 'Covid-Study-Data')))

mkdir /workspaces/capstone/Covid-Study-Data
STATUS:
 0 OUTPUT:
 


In [3]:
os.chdir('Covid-Study-Data')

#Create new dictionary 'tickers' containing the assets we want to examine and their corresponding Yahoo Finance ticker
tickers = {'EURUSD':'EURUSD=X','JPYUSD':'JPY=X','AUDUSD':'AUDUSD=X','CHFUSD':'CHF=X','CADUSD':'CAD=X','NOKUSD':'NOK=X',
 'SEKUSD':'SEK=X','NZDUSD':'NZDUSD=X','RUBUSD':'RUBUSD=X','CNYUSD':'CNY=X','MXNUSD':'MXN=X','GBPUSD':'GBPUSD=X',
 'BRLUSD':'BRL=X','DXY':'DX-Y.NYB','BTCUSD':'BTC-USD','GOLD':'GLD','S&P500':'^GSPC','DJIA':'^DJI','NASDAQ':'^IXIC'}

#Each value in dictionary 'Tickers' is now a dataframe for the daily prices of the respective asset
for key in tickers.keys():
    tickers[key] = yf.download(tickers[key], start=beginning, end=today, progress=False)[["Adj Close"]]
    
#Create dataframe 'Data' for the daily prices of all assets in this study
data = pd.concat(list(tickers.values()), axis=1)
data.columns = tickers.keys()

#Code for web-scraping daily treasury yields from the FRED website
baseurl = 'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id='
base2 = '&scale=left&cosd='
base3 = '&coed='
base4 = '&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date='
base5 = '&revision_date='
DGS10 = baseurl + 'DGS10' + base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=1962-01-02'
DFII10 = baseurl + 'DFII10'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=2003-01-02'
DGS30 = baseurl + 'DGS30'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=1977-02-15'
DFII30 = baseurl + 'DFII30'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=2010-02-22'
DGS2 = baseurl + 'DGS2'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=1976-06-01'

#Dictionary similar to 'Tickers', this time for treasury yields
datalinks = {'DGS10':DGS10,'DFII10':DFII10,'DGS30':DGS30,'DFII30':DFII30,'DGS2':DGS2}

/home/codespace/.python/current/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/codespace/.python/current/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/codespace/.python/current/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/codespace/.python/current/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex 

In [4]:
#Download CSV files for the treasury yield data
for element in datalinks.keys():
    destfile = os.path.join(os.getcwd(), element + '.csv')
    fromfile = datalinks.get(element)
    if not os.path.exists(destfile):
        urllib.request.urlretrieve(fromfile, destfile)
    else:
        print("'{}' already exists".format(destfile))

In [5]:
#Read the treasury CSV files into dataframes
DGS10 = pd.read_csv('DGS10.csv').set_index('DATE')
DFII10 = pd.read_csv('DFII10.csv').set_index('DATE')
DGS30 = pd.read_csv('DGS30.csv').set_index('DATE')
DFII30 = pd.read_csv('DFII30.csv').set_index('DATE')
DGS2 = pd.read_csv('DGS2.csv').set_index('DATE')

#Create dataframe 'Tdata' which contains daily yields for all treasuries
tdata = pd.concat([DGS10, DFII10,DGS30,DFII30,DGS2], axis=1)
tdata = tdata[tdata.DGS10 != '.']
tdata = tdata[tdata.DGS10 != ''].astype(float)

#Join treasury data with the daily prices of all other assets in 'data'
data = data.join(tdata)
data.dropna(inplace=True)
data.to_csv('Postcovid.csv')

In [6]:
#Go over the same process, but this time for the 'Pre-COVID' period
beginning = '2015-02-18'
ending = '2020-02-19'

tickers = {'EURUSD':'EURUSD=X','JPYUSD':'JPY=X','AUDUSD':'AUDUSD=X','CHFUSD':'CHF=X','CADUSD':'CAD=X','NOKUSD':'NOK=X',
 'SEKUSD':'SEK=X','NZDUSD':'NZDUSD=X','RUBUSD':'RUBUSD=X','CNYUSD':'CNY=X','MXNUSD':'MXN=X','GBPUSD':'GBPUSD=X',
 'BRLUSD':'BRL=X','DXY':'DX-Y.NYB','BTCUSD':'BTC-USD','GOLD':'GLD','S&P500':'^GSPC','DJIA':'^DJI','NASDAQ':'^IXIC'}

for key in tickers.keys():
    tickers[key] = yf.download(tickers[key], start=beginning, end=ending, progress=False)[["Adj Close"]]
    
data = pd.concat(list(tickers.values()), axis=1)
data.columns = tickers.keys()

baseurl = 'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id='
base2 = '&scale=left&cosd='
base3 = '&coed='
base4 = '&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date='
base5 = '&revision_date='
DGS10 = baseurl + 'DGS10' + base2 + beginning + base3 + ending + base4 + str(ending) + base5 + str(ending) + '&nd=1962-01-02'
DFII10 = baseurl + 'DFII10'+ base2 + beginning + base3 + ending + base4 + str(ending) + base5 + str(ending) + '&nd=2003-01-02'
DGS30 = baseurl + 'DGS30'+ base2 + beginning + base3 + ending + base4 + str(ending) + base5 + str(ending) + '&nd=1977-02-15'
DFII30 = baseurl + 'DFII30'+ base2 + beginning + base3 + ending + base4 + str(ending) + base5 + str(ending) + '&nd=2010-02-22'
DGS2 = baseurl + 'DGS2'+ base2 + beginning + base3 + ending + base4 + str(ending) + base5 + str(ending) + '&nd=1976-06-01'
datalinks = {'DGS10':DGS10,'DFII10':DFII10,'DGS30':DGS30,'DFII30':DFII30,'DGS2':DGS2}

for element in datalinks.keys():
    destfile = os.path.join(os.getcwd(), element + 'pre' + '.csv')
    fromfile = datalinks.get(element)
    if not os.path.exists(destfile):
        urllib.request.urlretrieve(fromfile, destfile)
    else:
        print("'{}' already exists".format(destfile))

/home/codespace/.python/current/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/codespace/.python/current/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/codespace/.python/current/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/codespace/.python/current/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex 

In [7]:
DGS10 = pd.read_csv('DGS10pre.csv').set_index('DATE')
DFII10 = pd.read_csv('DFII10pre.csv').set_index('DATE')
DGS30 = pd.read_csv('DGS30pre.csv').set_index('DATE')
DFII30 = pd.read_csv('DFII30pre.csv').set_index('DATE')
DGS2 = pd.read_csv('DGS2pre.csv').set_index('DATE')
tdata = pd.concat([DGS10, DFII10,DGS30,DFII30,DGS2], axis=1)
tdata = tdata[tdata.DGS10 != '.']
tdata = tdata[tdata.DGS10 != ''].astype(float)
data = data.join(tdata)
data.dropna(inplace=True)
data.to_csv('Precovid.csv')

In [8]:
#Go over the same process, but this time for the entire timeperiod from 2015-2020
beginning = '2015-02-18'
today = datetime.today().strftime('%Y-%m-%d')
tickers = {'EURUSD':'EURUSD=X','JPYUSD':'JPY=X','AUDUSD':'AUDUSD=X','CHFUSD':'CHF=X','CADUSD':'CAD=X','NOKUSD':'NOK=X',
 'SEKUSD':'SEK=X','NZDUSD':'NZDUSD=X','RUBUSD':'RUBUSD=X','CNYUSD':'CNY=X','MXNUSD':'MXN=X','GBPUSD':'GBPUSD=X',
 'BRLUSD':'BRL=X','DXY':'DX-Y.NYB','BTCUSD':'BTC-USD','GOLD':'GLD','S&P500':'^GSPC','DJIA':'^DJI','NASDAQ':'^IXIC'}

for key in tickers.keys():
    tickers[key] = yf.download(tickers[key], start=beginning, end=today, progress=False)[["Adj Close"]]
data = pd.concat(list(tickers.values()), axis=1)
data.columns = tickers.keys()

baseurl = 'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id='
base2 = '&scale=left&cosd='
base3 = '&coed='
base4 = '&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date='
base5 = '&revision_date='
DGS10 = baseurl + 'DGS10' + base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=1962-01-02'
DFII10 = baseurl + 'DFII10'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=2003-01-02'
DGS30 = baseurl + 'DGS30'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=1977-02-15'
DFII30 = baseurl + 'DFII30'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=2010-02-22'
DGS2 = baseurl + 'DGS2'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=1976-06-01'
datalinks = {'DGS10':DGS10,'DFII10':DFII10,'DGS30':DGS30,'DFII30':DFII30,'DGS2':DGS2}

for element in datalinks.keys():
    destfile = os.path.join(os.getcwd(), element + 'meta' + '.csv')
    fromfile = datalinks.get(element)
    urllib.request.urlretrieve(fromfile, destfile)
    
DGS10 = pd.read_csv('DGS10meta.csv').set_index('DATE')
DFII10 = pd.read_csv('DFII10meta.csv').set_index('DATE')
DGS30 = pd.read_csv('DGS30meta.csv').set_index('DATE')
DFII30 = pd.read_csv('DFII30meta.csv').set_index('DATE')
DGS2 = pd.read_csv('DGS2meta.csv').set_index('DATE')

tdata = pd.concat([DGS10, DFII10,DGS30,DFII30,DGS2], axis=1)
tdata = tdata[tdata.DGS10 != '.']
tdata = tdata[tdata.DGS10 != ''].astype(float)
data = data.join(tdata)
data.dropna(inplace=True)
data.to_csv('data.csv')

/home/codespace/.python/current/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/codespace/.python/current/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/codespace/.python/current/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/codespace/.python/current/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex 

In [2]:
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import pandas as pd
import os
import scipy.stats as stats            # Basic package for basic univariate regressions
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

/tmp/ipykernel_5765/1469471277.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
os.chdir('Covid-Study-Data')

#Read CSV files from the file created in 'Study-Data'
pre = pd.read_csv('Precovid.csv',index_col = [0],parse_dates = True)
post = pd.read_csv('Postcovid.csv',index_col = [0],parse_dates = True)
meta = pd.read_csv('data.csv',index_col = [0],parse_dates = True)

#Rename columns for US Treasuries
meta.rename(columns = {'DGS10':'US10Y','DGS2':'US2Y','DGS30':'US30Y'}, inplace = True)
pre.rename(columns = {'DGS10':'US10Y','DGS2':'US2Y','DGS30':'US30Y'}, inplace = True)
post.rename(columns = {'DGS10':'US10Y','DGS2':'US2Y','DGS30':'US30Y'}, inplace = True)

#Create new dataframes for daily percentage changes
prepct = pre.pct_change()[1:].astype(float)
postpct = post.pct_change()[1:].astype(float)
metapct = meta.pct_change()[1:].astype(float)

#Replace infinite percentage change and nan values with 0
postpct.replace([np.inf, -np.inf], 0,inplace=True)
prepct.replace([np.inf, -np.inf], 0,inplace=True)
metapct.replace([np.inf, -np.inf], 0,inplace=True)
postpct.replace([np.nan],0,inplace=True)
prepct.replace([np.nan],0,inplace=True)
metapct.replace([np.nan],0,inplace=True)

#x dataframes contain independent variables, y dataframes contain dependent variables
#xmeta is a dataframe containing daily percentage changes for all assets considered dependent variables
xmeta = metapct[['US2Y','US10Y','US30Y','DFII10','DFII30','S&P500','DJIA','NASDAQ','GOLD','BTCUSD']]
xmeta = xmeta[1:]
ymeta = metapct[['DXY','EURUSD','GBPUSD','CADUSD','CHFUSD','JPYUSD','AUDUSD','NZDUSD','NOKUSD','SEKUSD','MXNUSD','RUBUSD',
                'BRLUSD','CNYUSD','BTCUSD','GOLD']][1:]

#'pre' and 'post' correspond to the pre-COVID and post-COVID periods
xpost = postpct[['US2Y','US10Y','US30Y','DFII10','DFII30','S&P500','DJIA','NASDAQ','GOLD','BTCUSD']]
xpost = xpost[1:]
xpre = prepct[['US2Y','US10Y','US30Y','DFII10','DFII30','S&P500','DJIA','NASDAQ','GOLD','BTCUSD']]
xpre = xpre[1:]
ypre = prepct[['DXY','EURUSD','GBPUSD','CADUSD','CHFUSD','JPYUSD','AUDUSD','NZDUSD','NOKUSD','SEKUSD','MXNUSD','RUBUSD',
                'BRLUSD','CNYUSD','BTCUSD','GOLD']][1:]
ypost = postpct[['DXY','EURUSD','GBPUSD','CADUSD','CHFUSD','JPYUSD','AUDUSD','NZDUSD','NOKUSD','SEKUSD','MXNUSD','RUBUSD',
                'BRLUSD','CNYUSD','BTCUSD','GOLD']][1:]

FileNotFoundError: [Errno 2] No such file or directory: 'Covid-Study-Data'